In [3]:
# Author: Vamshi Chenna - Week 1
# Revised by: Shiva Deviah - Week 3
import pandas as pd
import re

pattern = re.compile(r'Restaurant|Alcohol')
def is_valid_attr(dct):
    return any(pattern.match(k) for k in dct.keys())

business_json_df = (pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
                      .dropna(subset=['attributes']))

#Get the restuarant Data from the businesses
mask = (business_json_df['attributes'].map(is_valid_attr) 
        | business_json_df['categories'].str.contains(r"Food|Restaurant|Bar", case=False) 
        | business_json_df['name'].str.contains("Restaurant|Cuisine", case=False))
business_json_df = business_json_df[mask]

business_json_df.shape

(113202, 15)

In [6]:
attrDF = pd.DataFrame(business_json_df.attributes.dropna().tolist())
#Gets the individual count for every attribute
attrDF.count()

AcceptsInsurance                2811
AgesAllowed                      396
Alcohol                        47892
Ambience                       47569
BYOB                             911
BYOBCorkage                     1409
BestNights                      6842
BikeParking                    78905
BusinessAcceptsBitcoin          7594
BusinessAcceptsCreditCards    101443
BusinessParking                96813
ByAppointmentOnly              20214
Caters                         40038
CoatCheck                       8527
Corkage                          657
DietaryRestrictions              138
DogsAllowed                    12490
DriveThru                       6754
GoodForDancing                  9158
GoodForKids                    56618
GoodForMeal                    47483
HairSpecializesIn               1683
HappyHour                       9279
HasTV                          47527
Music                           8803
NoiseLevel                     43704
Open24Hours                      350
O

In [9]:
#To Caliculate averages and attribute density,
attrDensity = attrDF.count(axis=1)

#Gets the number of business having attributes more than 1
attrDensity[attrDensity>1].size

111767

In [10]:
#Get the average of attributes per business

#Avergage number of attributes per Resturant : 11.338933217616168
#Resturants which have atlest one attribute : 112919
#Restuarants having more than one attribute : 111549
#Resturants having more than three attributes : 101870
#Resturants having more than five attributes : 76988
#Resturants having more than ten attributes : 50864

attrDensity.mean()

11.320171021713398

In [13]:
#Categories-----------------------------------------------------------------------------------------
categories = business_json_df.categories.dropna().str.split(r'\s*,\s*', expand=True)
#Get the categories count per user
catcount = categories.count(axis=1)

catcount.head()

0    6
1    9
2    5
5    2
6    2
dtype: int64

In [14]:
#Get the mean of the no of categories
categories.count(axis=1).mean()

4.043264260151107

In [15]:
#Gets the number of business having categories more than 1
catcount[catcount>1].size

112649

In [16]:
#Average category per resturant : 4.0274109643857541
#Restuarants having atlest one category : 114954
#Restuarants having more than one category : 114390
#Restuarants having more than two categories : 82534

#Shopping	24503
#Beauty & Spas	15956
#Nightlife	12147
#Bars	10853
#Coffee & Tea	6936


#Reviews-----------------------------------------------------------------------------------------

#Average No of Reviews for a busineses : 43.107634782608699
#Minimum no of reviews for a resturant : 3

#Restuarants having atlest 10 reviews : 65719
#Restuarants having more than 15 reviews : 52183
#Restuarants having more than 20 reviews : 42111
#Resturants having more than 25 reviews : 36274
#Resturants having more than 30 reviews : 31881


business_json_df.review_count.mean()

43.72633875726577